In [1]:
import pandas as pd
import numpy as np
import warnings
import pickle
from math import sqrt
warnings.filterwarnings('ignore')
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

In [2]:
# Connect to Workspace
# Enter details of your AML workspace
subscription_id = "4a571c1c-a483-4a43-9930-490479d70db0"
resource_group = "Learn_MLOps"
workspace = "MLOs_WS"

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace)

## Download scaler and model from workspace

In [4]:
scaler = ml_client.models.download(name='scaler',version='1')

In [5]:
svc_model = ml_client.models.download(name='support-vector-classifier',version='1')

## Load files

In [6]:
with open('./scaler/scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [7]:
# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy
sess = rt.InferenceSession("./support-vector-classifier/svc.onnx")

In [8]:
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

## inference on test data

In [9]:
test_data = np.array([34.927778, 0.24, 7.3899, 83, 16.1000, 1])

In [10]:
# Scale data
test_data = scaler.fit_transform(test_data.reshape(1, 6))

In [11]:
pred_onx = sess.run([label_name], {input_name: test_data.astype(numpy.float32)})[0]

In [12]:
pred_onx[0]

1